# Systemy uczące się - Zad. dom. 1: Minimalizacja ryzyka empirycznego

### Autor rozwiązania: Maciej Wieczorek, 148141

Celem zadania jest zaimplementowanie własnego drzewa decyzyjnego wykorzystującego idee minimalizacji ryzyka empirycznego. 

## Twoja implementacja

Twoim celem jest uzupełnić poniższą klasę `TreeNode` tak by po wywołaniu `TreeNode.fit` tworzone było drzewo decyzyjne minimalizujące ryzyko empiryczne. Drzewo powinno wspierać problem klasyfikacji wieloklasowej (jak w przykładzie poniżej). Zaimplementowany algorytm nie musi (ale może) być analogiczny do zaprezentowanego na zajęciach algorytmu dla klasyfikacji. Wszelkie przejawy inwencji twórczej wskazane. Pozostaw komenatrze w kodzie, które wyjaśniają Twoje rozwiązanie.

Schemat oceniania:
- wynik na ukrytym zbiorze testowym (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u 1 +20%,
- wynik na ukrytym zbiorze testowym (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u 2 +40%,
- wynik na ukrytym zbiorze testowym (automatyczna ewaluacja) celność klasyfikacji >= bardziej zaawansowanego baseline'u 3 +40%.

Niedozwolone jest korzystanie z zewnętrznych bibliotek do tworzenia drzewa decyzyjnego (np. scikit-learn). 
Możesz jedynie korzystać z biblioteki numpy.

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod), o ile będzie się w nim na koniec znajdowała kompletna implementacja klasy `TreeNode` w jednej komórce.

In [60]:
import numpy as np

def entropy(X):
	_, value_counts = np.unique(X, return_counts=True)
	proba = value_counts / len(X)

	return -np.sum(proba * np.log2(proba))

def measure_split_value(target, index):
	target_l = target[:index+1]
	target_r = target[index+1:]
	entropy_l = entropy(target_l) * (len(target_l) / len(target))
	entropy_r = entropy(target_r) * (len(target_r) / len(target))
	info_gain = entropy(target) - (entropy_l + entropy_r)
	return info_gain 

def split_data(data, target, split):
	l_indices = np.where(data[:, split[0]] <= split[1])[0]
	r_indices = np.where(data[:, split[0]] > split[1])[0]
	return (data[l_indices], target[l_indices], data[r_indices], target[r_indices])


class TreeNode(object):
	def __init__(self):
		self.left = None # Typ: Node, wierzchołek znajdujący się po lewej stornie
		self.right = None # Typ: Node, wierzchołek znajdujący się po prawej stornie
		self.value = None # klasa decyzyjna liścia
		self.best_split = None # warunek podziału, para: (indeks atrybutu, wartość na atrybucie)
		
	def fit(self, data, target):
		"""
		Argumenty:
		data -- numpy.ndarray, macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech
		target -- numpy.ndarray, wektor klas o długości n, gdzie n to liczba przykładów
		"""
		best_split_gain = 0
		for i in range(data.shape[1]):
			attr_data = data[:,i]
			sorted_indices = np.argsort(attr_data)
			sorted_attr = attr_data[sorted_indices]
			sorted_target = target[sorted_indices]

			# sprawdź podziały jeśli pomiędzy x_i a x_i+1 jest różnica w klasie decyzyjnej i wartości
			splits = np.intersect1d(np.where(sorted_target[:-1] != sorted_target[1:])[0], np.where(sorted_attr[:-1] != sorted_attr[1:])[0])
			for split_index in splits:
				split_gain = measure_split_value(sorted_target, split_index)
				if split_gain > best_split_gain:
					best_split_gain = split_gain
					self.best_split = (i, sorted_attr[split_index]) # (atrybut, wartość podziału <= na lewo)

		if self.best_split is not None:
			data_l, target_l, data_r, target_r = split_data(data, target, self.best_split)
			if (len(data_l) == 0 or len(data_r) == 0):
				print()
				raise Exception
			self.left = TreeNode()
			self.right = TreeNode()
			self.left.fit(data_l, target_l)
			self.right.fit(data_r, target_r)
		else:
			self.value = target[0]
		

	
	def predict(self, data):
		"""
		Argumenty:
		data -- numpy.ndarray, macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech

		Wartość zwracana:
		numpy.ndarray, wektor przewidzoanych klas o długości n, gdzie n to liczba przykładów
		"""
		y_pred = np.zeros(data.shape[0])
		for i, x in enumerate(data):
			node = self
			while node and node.value is None:
				if x[node.best_split[0]] > node.best_split[1]:
					node = node.right
				else:
					node = node.left
			if node.value:
				y_pred[i] = node.value
		return y_pred


## Przykład trenowanie i testowania drzewa
 
Później znajduje się przykład trenowania i testowania drzewa na zbiorze danych `iris`, który zawierający 150 próbek irysów, z czego każda próbka zawiera 4 atrybuty: długość i szerokość płatków oraz długość i szerokość działki kielicha. Każda próbka należy do jednej z trzech klas: `setosa`, `versicolor` lub `virginica`, które są zakodowane jak int.

Możesz go wykorzystać do testowania swojej implementacji. Możesz też zaimplementować własne testy lub użyć innych zbiorów danych, np. innych [zbiorów danych z scikit-learn](https://scikit-learn.org/stable/datasets/toy_dataset.html#toy-datasets).

In [59]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=2042)

tree_model = TreeNode()
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.88
